In [1]:
%load_ext autoreload
%autoreload 2
# Add parent directory into system path
import sys, os
sys.path.insert(1, os.path.abspath(os.path.normpath('..')))

import torch
from torch import nn
from torch.nn.init import calculate_gain

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f'CUDA {i}: {torch.cuda.get_device_name(i)}')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = 'cpu'
torch.set_default_dtype(torch.float32)

CUDA 0: NVIDIA GeForce GTX 1650 Ti


In [2]:
from models.MLP import Davies2021
net = Davies2021(N_layers=8, width=32, activation=nn.Softplus(30), last_activation=nn.Softplus(30)).to(device)
#net = Davies2021(N_layers=8, width=28, activation=nn.SiLU(), last_activation=nn.Identity()).to(device)


In [3]:
import os
from utils.dataset_generator import ImplicitDataset

dataset_name = '../datasets/box_1f0_gyroid_4pi'
output_stl = dataset_name+'.stl'
net.load_state_dict(torch.load(f'{dataset_name}-MLP.pth'))

[pyigl_import] module igl not found. trying to import pyigl


In [10]:
from utils.dataset_generator import run_batch
from kaolin.metrics import voxelgrid
import math
def test_iou(net, x, true_sdf, batch_size=10000):
    assert hasattr(net, 'predict_test'), 'nn.Module must has predict_test function, i.e. extending from Base'
    # predict sdf from net
    predict_sdf = run_batch(net.predict_test, x, batch_size=batch_size)
    # convert list to voxel grid
    N = x.shape[0]
    Nx = math.ceil(N**(1/3))
    voxelgrid_ground = true_sdf.reshape((1, Nx, Nx, Nx))
    voxelgrid_pred = predict_sdf.reshape((1, Nx, Nx, Nx))
    return voxelgrid.iou(voxelgrid_pred, voxelgrid_ground)

In [11]:
test_iou(net, test_dataset.uniform.points, test_dataset.uniform.sdfs)

tensor([0.9848], device='cuda:0')